In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [4]:
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
features = newdf.to_numpy()
emissions = df['CO2E'].to_numpy()

In [119]:
def modelScore(X, y, model):
    avgR2 = avgSSE = avgMAPE = 0
    #cross validation using K folds
    kFolds = 5
    for i, (trainIndex, testIndex) in enumerate(KFold(n_splits=kFolds, shuffle=True, random_state=42).split(X)):
        X_train = X[trainIndex]
        X_test = X[testIndex]
        y_train = y[trainIndex]
        y_test = y[testIndex]
    

        if model == 'linear regression':
            fittedModel = LinearRegression().fit(X_train, y_train)
            
        elif model == 'decision tree':
            dtModel = tree.DecisionTreeRegressor()
            fittedModel = dtModel.fit(X_train, y_train)
                
        elif model == 'random forest':
            y_train = y_train.flatten()
            RF = RandomForestRegressor(max_features='log2', n_estimators=200, max_depth=20)
            fittedModel = RF.fit(X_train, y_train)
            
        elif model == 'support vector regression':
            y_train = y_train.flatten()
            fittedModel = SVR(kernel = 'linear', C=1).fit(X_train, y_train)
    
        elif model == 'polynomial regression':
            polyFeatures = PolynomialFeatures(degree=3, interaction_only=True, include_bias=True, order='C')
            polyX_train = polyFeatures.fit_transform(X_train)
            polyX_test = polyFeatures.transform(X_test)
            fittedModel = LinearRegression().fit(polyX_train, y_train)
            X_test = polyX_test

        elif model == 'xgboost':
            xgbModel = xgb.XGBModel(learning_rate=0.225, n_estimators=225, booster='gblinear',
                                        reg_alpha=0, reg_lambda=0, eval_metric='auc', 
                                        objective='reg:squaredlogerror', updater='coord_descent', 
                                        feature_selector='thrifty')
            fittedModel = xgbModel.fit(X_train, y_train)
        
        elif model =='lasso':
            lassoModel = Lasso(tol=.0001, max_iter=2000, selection='random', alpha=.15)

            fittedModel = lassoModel.fit(X_train, y_train)
        
        elif model =="neural network":
            nn = MLPRegressor(alpha=0.001, batch_size=25, hidden_layer_sizes=(10, 10, 10, 10),
             learning_rate_init=0.05, max_iter=5000, random_state=42, tol=1e-1, max_fun=25000,
             solver='lbfgs', activation='identity')

            fittedModel = nn.fit(X_train, y_train)

        y_pred = fittedModel.predict(X_test)

        #scores
        avgR2 += r2_score(y_test, y_pred)    
        avgSSE += np.sum((y_test - y_pred) ** 2)
        avgMAPE += mean_absolute_percentage_error(y_test, y_pred)



    avgR2, avgSSE, avgMAPE = avgR2 / kFolds, avgSSE / kFolds, avgMAPE / kFolds
   #print results
    print(model)
    print("R2 = " + str(avgR2))
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))




In [120]:
modelScore(features, emissions, 'neural network')

neural network
R2 = 0.9158294656879136
SSE = 166.71653292690127
MAPE = 0.04583188851292793


In [10]:
modelScore(features, emissions, 'linear regression')

linear regression
R2 = 0.9132785915590607
SSE = 172.6195696345183
MAPE = 0.04614202622279183


In [21]:
modelScore(features, emissions, 'decision tree')

decision tree
R2 = 0.6662928948462766
SSE = 712.8
MAPE = 0.08939441482941882


In [8]:
modelScore(features, emissions, 'random forest')

random forest
R2 = 0.7271477493885369
SSE = 642.307705
MAPE = 0.08345888361923537


In [9]:
modelScore(features, emissions, 'support vector regression')

support vector regression
R2 = 0.8783182219644387
SSE = 231.72982845335702
MAPE = 0.0474385532925264


In [66]:
modelScore(features, emissions, 'polynomial regression')

polynomial regression
R2 = 0.9176046244206117
SSE = 213.04590046008312
MAPE = 0.030430745629900446
[-1.69194699e-17  3.62595378e-10  4.56798766e-08  1.97123915e-09
 -1.11341422e-17  1.13833632e-09  2.92835991e-09  2.02696360e-08
  6.73336891e-10  6.17039547e-10  8.66452411e-10  1.16878054e-08
 -2.90296463e-07  2.94382497e-07  1.14371365e-08  2.17557169e-09
  7.16152771e-09  1.72713180e-08  1.80761080e-07  4.82485226e-09
  7.25122937e-09  5.18411764e-09  6.99343498e-08 -7.99327597e-07
 -6.02403273e-07  2.74079260e-07 -1.99525229e-07  8.66979788e-07
  6.16049390e-06  2.78413082e-06  5.57426197e-07 -1.18977755e-07
  1.36566543e-06 -1.22274176e-06  1.18274350e-08  1.33885959e-08
  3.14010097e-08  3.68003228e-07  4.05805057e-08  2.25445801e-08
  8.37582956e-09  1.39980170e-07  1.07391380e-06  6.83001822e-09
  1.75701592e-08  1.21617816e-07  4.04002122e-09  3.70223741e-09
  5.19871449e-09  7.01268325e-08 -1.74177878e-06  2.27789589e-08
  2.47142970e-07  2.34695280e-08  1.35753867e-08  6.0735

: 

In [11]:
modelScore(features, emissions, 'xgboost')

xgboost
R2 = 0.9073441886813531
SSE = 195.56138811718557
MAPE = 0.04682619770749446


In [12]:
modelScore(features, emissions, 'lasso')

lasso
R2 = 0.9012026099356827
SSE = 199.96046739839318
MAPE = 0.049636892695834094


In [13]:
modelsList = ['linear regression', 'decision tree', 'random forest', 'support vector regression', 'polynomial regression', 'xgboost', 'lasso']
multipleFeatures = df[['DISPLAY (inch)', 'STORAGE (gb)', 'MEMORY (gb)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', 'number of camera (back)', 'Sum of megapixels', 'MAIN CAMERA (mp)']].to_numpy()
featuresList = ['DISPLAY (inch)', 'STORAGE (gb)', 'MEMORY (gb)', 'CPU (cores)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', 'BATTERY (mAh)', 'number of camera (back)', 'Sum of megapixels', 'MAIN CAMERA (mp)']

In [14]:
for model in modelsList:
    modelScore(multipleFeatures, emissions, model)

linear regression
R2 = 0.9157485339743221
SSE = 166.44599819781578
MAPE = 0.045766455386079326
decision tree
R2 = 0.7772862077161674
SSE = 465.4
MAPE = 0.0735462453987529
random forest
R2 = 0.776251396462398
SSE = 509.93691499999994
MAPE = 0.07660573835582944


KeyboardInterrupt: 

In [ ]:
"""for x in featuresList:
    print(x)
    for model in modelsList:
        print(modelScore(df[x].to_numpy().reshape(-1, 1), emissions, model))
    print("")  """

'for x in featuresList:\n    print(x)\n    for model in modelsList:\n        print(modelScore(df[x].to_numpy().reshape(-1, 1), emissions, model))\n    print("")  '